moves are quantum gates
- OBJECTIVE: PLAYER GETS HIGHEST PROBABILITY OF GETTING TO THE 1 STATE
- moves manipulate qubit state
- negative phases are defenses (probably not ?)
- record moves and apply gates to a circuit
- at the end of the game, measure and see who wins
- EXTRA feature: playback: see what moves have been done already
- make room to add more cards
- cards with turn cooldown
Setting up game:
- set up enemy and player moves (are cards that are given prechosen or are they chosen randomly from a predecided array)
- text in top corner showing current vector state of qubit (backend result)
- game start: define cards that the player has
- load up cardUI with # of cards that are in the player array
- function declareWinner

- function measureAndDisplay
    - 1) one by one, send values through a circuit and measure
    - 2) says that it's loading as values run through ionq
    - 3) updates UI probability, vector state, and winning texts
    - 4) returns 1 if player is winning, 0 if player is losing
- function player's move:
    - 1) waits for card user input. if card is on cooldown then deny and repeat this step.
    - 2) return selected value and shrink the selected card to 50
    - 3) puts first value in a list
        ONLY IF PLAY 2 CARDS HAS BEEN SELECTED: 
        - 5) wait for another user response
        - 6) shrink card that was selected to 50
        - 7) puts second value in the list
    - 8) call measuring function
    - 9) call enemyTurn and subtract remaining turns by 1
- function enemy's move:
    - 0) if winning turn number is zero then call declareWinner function and return absolutely nothing
    - 1) #all optimization logic here
    - n-1) call measuring function
    - n) call playerTurn
- how will winning states be declared?
    - 1) if you get |1> you win

In [140]:
import logging #is this necessary? who knows
import math
import sys
import os
import time
import random

sys.path.insert(0, os.path.abspath('./..'))
import jupylet.color
from jupylet.sprite import Sprite
from jupylet.label import Label
from jupylet.app import App
from jupylet.state import State
from jupylet.collision import trbl, hitmap_and_outline_from_alpha, compute_collisions 

from qiskit import *
import numpy as np
import math

In [141]:
app = App()

In [142]:
# V A R I A B L E S

possibleGatesNames = ("X Gate", "SX Gate", "Z Gate", "Y Gate", "T Gate", "P PI/2", "P PI/4", "P PI") #can enter them as symbols as well
possibleGatesDict = { #FILL THIS OUT (whole game depends on it)
    possibleGatesNames[0] : QuantumCircuit(0).x(0),
    possibleGatesNames[1] : QuantumCircuit(0).rx(math.pi/2, 0),
    possibleGatesNames[2] : QuantumCircuit(0).z(0),
    possibleGatesNames[3] : QuantumCircuit(0).y(0),
    possibleGatesNames[4] : QuantumCircuit(0).t(0),
    possibleGatesNames[5] : QuantumCircuit(0).p(math.pi/2, 0),
    possibleGatesNames[6] : QuantumCircuit(0).p(math.pi/4, 0),
    possibleGatesNames[7] : QuantumCircuit(0).p(math.pi, 0)
}

turnNumber = 10 #this can be changed

availableCards = [possibleGatesDict[possibleGatesNames[0]],possibleGatesDict[possibleGatesNames[1]],possibleGatesDict[possibleGatesNames[2]],possibleGatesDict[possibleGatesNames[3]]] 

#INSERT YOUR GATES *FROM DICTIONARY HERE* (whole game depends on it)
#example: how to set the first card that will be in each hand to the second possible gate 
#availableCards = [possibleGatesDict.possibleGatesNames[1]]
availableCardCooldowns = [1,3,4,2]
#INSERT how many turns per gate it'll take for the player to unlock card per gate here

#player card stats
cardsUI = []
cardsText = []
cardsStates = [] #0 state is not in cooldown
selectedCardNum = 0

enemySelectedCardNum = 0
enemyCardsStates = []
enemyCardsText = []

numberOfCards = len(availableCards)

playerHand = availableCards.copy()
enemyHand = availableCards.copy()

currentCircuitChosen = 0

winning = "Nobody is Currently Winning!"
currentQubitState = "no qubit vector"
currentQubitProbability = "|1> Probability: N/A"
turn = "You Go"



In [143]:
# S T A T E S

state = State(
    cardChosen = False,
    shrinkMe = False,
    isPlayerTurn = True,
    gameStarted = False,
    gameOver = False,
)


In [144]:
# S P R I T E S

playerFace = Sprite('playerDummySprite.png', x=80, y=app.height-80, scale=1)
enemyFace = Sprite('enemyDummySprite.png', x=app.width-80, y=app.height-80, scale=1)
cursor = Sprite('cursorDummySprite.png', scale = 0.1, collisions = True)

stateText = Label(currentQubitState, color='cyan', font_size=15, x=app.width/2, y=app.height-40, anchor_x='right')
possibleText = Label(currentQubitProbability, color='cyan', font_size=15, x=app.width/2, y=app.height-20, anchor_x='right')
winningText = Label(winning, color ='cyan', font_size=25, x=(app.width-90), y=app.height-60, anchor_x='right')
turnText = Label(turn, color ='cyan', font_size=10, x=(app.width/2), y=app.height-80, anchor_x='right')

logLabel = Label('foo', color = 'white', font_size=10, x=(app.width/2), y=app.height-100, anchor_x='right')

In [145]:
# C A R D  S E T U P

for i in range(numberOfCards):
    #cardsUI.append(Card(playerHand[i], False))
    cardsUI.append(Sprite('cardDummySprite.png', x = (60*i) + 30, y=app.width/2, scale=1,collisions=True))
    
    dictVal = list(possibleGatesDict.keys())[list(possibleGatesDict.values()).index(playerHand[i])]
    enDictVal = list(possibleGatesDict.keys())[list(possibleGatesDict.values()).index(enemyHand[i])]
    cardsText.append(Label(dictVal, font_size=(9*round(cardsUI[i].scale)), color = 'cyan', x=cardsUI[i].x-20, y=cardsUI[i].y))
    enemyCardsText.append(enDictVal) 
    cardsStates.append(0)
    enemyCardsStates.append(0)
    #cardsUI[i].spr.x += (i*32) + 4

<h1>FUNCTIONS<h1>

In [146]:
#print(cardsUI[1])

In [147]:
#where render was

In [148]:
def isColliding(sprite, o, debug=False):
    #jupylet source code adjustment: custom boolean returning function

    #if sprite.distance_to(o) > sprite.radius + o.radius:
    #    return

    x0, y0 = sprite.position.xy
    x1, y1 = o.position.xy

    t0, r0, b0, l0 = sprite._trbl()
    t1, r1, b1, l1 = o._trbl()

    if t0 + y0 < b1 + y1 or t1 + y1 < b0 + y0:
        return False

    if r0 + x0 < l1 + x1 or r1 + x1 < l0 + x0:
        return False

    #return compute_collisions(o, sprite, debug=debug)
    return True


In [149]:
def measureAndDisplay(): #use currentCircuitChosen!
    """
    main idea:
    - 1) one by one, send values through a circuit and measure
    - 2) says that it's loading as values run through ionq
    - 3) updates UI probability, vector state, and winning texts
    - 4) returns 1 if player is winning, 0 if player is losing
    """
    #to change text: declaredLable.text = str(intValue)

In [150]:
@app.run_me_every(1/2)
def gameController(ct, dt):
    pass
        

In [151]:
def declareWinner():
    if measureAndDisplay() == 1:
        winningText.text = "YOU WIN!"
    elif measureAndDisplay() == 0:
        winningText.text = "You lost. Go try again!"
    state.gameStarted = False

In [152]:
def cooldown(array,arrayUI=[]):
    for i in range(len(array)):
        if array[i] > 0:
            array[i] -= 1
            if array[i] == 0 and arrayUI:
                arrayUI[i].angle = 0
                logLabel.text = "Some cards came off cooldown that you can use!"

In [153]:
def enemyTurn(): #for now, it chooses a random gate. If available, comment out below and put code optimization here
#if not state.isPlayerTurn and not state.gameOver:
    turn = "They Go"
    turnText.text = turn
    state.PlayerJustWent = False
    #logLabel.text = "enemyrun"
    tempChoice = random.randint(0,(len(enemyHand)-1))
    #below, whichever way you choose a card, make sure to get the index of the chosen card 
    #so enemy can check if it's on cooldown:
    if enemyCardsStates[tempChoice] > 0:
        while enemyCardsStates[tempChoice] > 0:
            tempChoice = random.randint(0,(len(enemyHand)-1))
    enemySelectedCardNum = tempChoice
    #find selected card
    tempCard = 0
    for i in range(len(possibleGatesNames)):
        if enemyCardsText[enemySelectedCardNum] == possibleGatesNames[i]:
            tempCard = i
    currentCircuitChosen = possibleGatesDict[possibleGatesNames[tempCard]]        
    cooldown(enemyCardsStates)
    enemyCardsStates[enemySelectedCardNum] = availableCardCooldowns[enemySelectedCardNum]
    time.sleep(0.5)
    turn = "You Go"
    logLabel.text = "en card "+str(enemySelectedCardNum)
    turnText.text = turn
    #state.aTurnIsHappening = False
    state.isPlayerTurn = True
    #state.needNextTurn = True
    #pass

In [154]:
def prepareForEnemyTurn(tnum):
    cooldown(cardsStates,cardsUI)
    tnum = tnum - 1
    turn = "They Go"
    turnText.text = "They Go"
    state.isPlayerTurn = False
    state.aTurnIsHappening = False
    state.playerJustWent = True
    state.needNextTurn = True

In [155]:
def updateCards():
    pass

In [156]:
def updateUI():
    pass

MORE EVENTS THAT CONSTANTLY RUN GO DOWN HERE

In [157]:
#@app.run_me_every(1/20)
def playerTurn():
    #find selected card
    tempCard = 0
    for i in range(len(possibleGatesNames)):
        if cardsText[selectedCardNum] == possibleGatesNames[i]:
            tempCard = i
    currentCircuitChosen = possibleGatesDict[possibleGatesNames[tempCard]]
    # IMPORTANT: measure function here!
    prepareForEnemyTurn(turnNumber)
    logLabel.text = "enemyGoing"
    enemyTurn()
    """
    - 1) waits for cardChosen to be true. if card is on cooldown then deny and repeat this step.
    - 2) return selected value and shrink the selected card to 50
    - 3) puts first value in a list
        ONLY IF PLAY 2 CARDS HAS BEEN SELECTED: 
        - 5) wait for another user response
        - 6) shrink card that was selected to 50
        - 7) puts second value in the list
    - 8) call measuring function
    - 9) subtract remaining turns by 1, call lessenCooldowns(cardsStates) function
    - 9) call enemyTurn and subtract turnNumber by 1
    """

In [158]:
"""def cardShrink(ang):
    ang = 90; #timeout/sleep?"""

'def cardShrink(ang):\n    ang = 90; #timeout/sleep?'

In [159]:
@app.event
def mouse_release_event(x, y, button):
    #works here
    if state.isPlayerTurn:
        #works here
        for i in range(len(cardsUI)):
            if isColliding(cursor,cardsUI[i]) and cardsStates[i] == 0:
            #if cursor.collisions_with(any(cardsUI)) and cardsStates[k] == 0:
            #if cursor.collisions_with(cardsUI[k]) and cardsStates[k] == 0:
            #if cursor.distance_to(cardsUI[k]) < cardsUI[k].radius:
            #if (cursor.x >= cardsUI[k].left) and (cursor.x <= cardsUI[k].right) and (cursor.y >= cardsUI[k].bottom) and (cursor.y <= cardsUI[k].top) and (cardsStates[k] == 0):
                selectedCardNum = i
                logLabel.text= str(selectedCardNum)
                #logLabel.text = str(cursor.distance_to(cardsUI[k]))
                state.cardChosen = True
                state.shrinkMe = True
                cardsUI[i].angle = 90
                state.shrinkMe = False
                cardsStates[i] = availableCardCooldowns[i]
                #logLabel.text='looping'+str(k)
            elif cardsStates[i] > 0 or cardsUI[i].angle > 90:
                logLabel.text = "This card is still cooling down!"
        if state.cardChosen:
            playerTurn()
    else:
        logLabel.text="Not Your Turn!"

In [160]:
@app.event
def mouse_position_event(x, y, dx, dy):
    cursor.x = x
    cursor.y = y

In [161]:

@app.run_me_every(1/120)
def GAMEPLAY(ct,dt):
    if turnNumber < 1:
        state.isPlayerTurn = False
        state.gameOver = True
        declareWinner()
"""    

if not state.aTurnIsHappening and state.needNextTurn:
        if state.isPlayerTurn:
            playerTurn()
            if state.cardChosen:
                state.needNextTurn = False
                state.aTurnIsHappening = True
        #if not state.isPlayerTurn:
        if state.playerJustWent:
            logLabel.text = "enemy is runn"
            enemyTurn()
            state.needNextTurn = False
            state.aTurnIsHappening = True
        if turnNumber < 1:
            state.isPlayerTurn = False
            state.gameOver = True
            declareWinner()"""

'    \n\nif not state.aTurnIsHappening and state.needNextTurn:\n        if state.isPlayerTurn:\n            playerTurn()\n            if state.cardChosen:\n                state.needNextTurn = False\n                state.aTurnIsHappening = True\n        #if not state.isPlayerTurn:\n        if state.playerJustWent:\n            logLabel.text = "enemy is runn"\n            enemyTurn()\n            state.needNextTurn = False\n            state.aTurnIsHappening = True\n        if turnNumber < 1:\n            state.isPlayerTurn = False\n            state.gameOver = True\n            declareWinner()'

In [162]:
@app.event
def render(ct,dt):
    app.window.clear()
    for j in range (len(cardsUI)):
        cardsUI[j].draw()
        cardsText[j].draw()    
    playerFace.draw()
    enemyFace.draw()
    stateText.draw()
    possibleText.draw()
    winningText.draw()
    turnText.draw()
    logLabel.draw()
    cursor.draw()

In [163]:
#playerTurn()
app.run()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

In [164]:
app.get_logging_widget()

Output(layout=Layout(height='256px', overflow_y='scroll'))

In [165]:
#w.clear_output()


In [166]:
print(currentCircuitChosen)

0
